In [1]:
import gensim
import pandas as pd
from collections import Counter
from estnltk import Text
from ast import literal_eval

## Data

In [2]:
df=pd.read_csv('data/interim/stenos_cleaned_topics.csv')
df.shape

(209949, 22)

## Keep only text column

In [3]:
df.head()

heading  \
0                                                                                                                               10:00  Istungi rakendamine   
1  1.\n                        10:02  Eesti Keskerakonna fraktsiooni algatatud olulise tähtsusega riikliku küsimuse "Maksusüsteem – kuidas edasi?" arutelu   
2  1.\n                        10:02  Eesti Keskerakonna fraktsiooni algatatud olulise tähtsusega riikliku küsimuse "Maksusüsteem – kuidas edasi?" arutelu   
3  1.\n                        10:02  Eesti Keskerakonna fraktsiooni algatatud olulise tähtsusega riikliku küsimuse "Maksusüsteem – kuidas edasi?" arutelu   
4  1.\n                        10:02  Eesti Keskerakonna fraktsiooni algatatud olulise tähtsusega riikliku küsimuse "Maksusüsteem – kuidas edasi?" arutelu   

                  speaker                                          link_video  \
0  Aseesimees Siim Kallas     https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=0   
1  Aseesimees Siim Kallas   https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=125   
2          Kersti Sarapuu   https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=238   
3  Aseesimees Siim Kallas  https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=1194   
4             Aivar Sõerd  https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=1231   

                                       link_steno   index_pk   index_snd  \
0  https://stenogrammid.riigikogu.ee/201911191000  PKP-25040  SND-559399   
1  https://stenogrammid.riigikogu.ee/201911191000  PKP-25041  SND-559400   
2  https://stenogrammid.riigikogu.ee/201911191000  PKP-25041  SND-559401   
3  https://stenogrammid.riigikogu.ee/201911191000  PKP-25041  SND-559402   
4  https://stenogrammid.riigikogu.ee/201911191000  PKP-25041  SND-559403   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [3]:
df.columns

Index(['heading', 'speaker', 'link_video', 'link_steno', 'index_pk',
       'index_snd', 'text', 'year', 'month', 'day', 'time',
       'seconds_from_start', 'video_link_wo_time', 'text_wo_punct', 'date',
       'koosseis', 'fation', 'topic_id', 'topic_words', 'topic_cluster',
       'topic_cluster_sub', 'topic_words3'],
      dtype='object')

In [4]:
texts=df.text

## Clean and tokenize

In [5]:
def toks_postags_text(text):
    analysis = Text(text.lower()).tag_layer(["morph_analysis"])
    tokens=analysis.morph_analysis.text
    postags=[tag[0][0] for tag in analysis.morph_analysis.partofspeech.amb_attr_tuple_list]
    return tokens, postags

In [65]:
toks_postags_text('see on nii')

(['see', 'on', 'nii'], ['P', 'V', 'D'])

In [66]:
def toks_postags_texts(texts, print_every=10000):
    postags=[]
    toks=[]
    
    for i, text in enumerate(texts):
        if i%print_every==0:
            print(f'working on text no {i}')
        toks_, tags_ = toks_postags_text(text)
        assert len(toks_)==len(tags_)
        postags.append(tags_)
        toks.append(toks_)
    return toks, postags

In [67]:
%time tokens, postags = toks_postags_texts(texts)

working on text no 0
working on text no 10000
working on text no 20000
working on text no 30000
working on text no 40000
working on text no 50000
working on text no 60000
working on text no 70000
working on text no 80000
working on text no 90000
working on text no 100000
working on text no 110000
working on text no 120000
working on text no 130000
working on text no 140000
working on text no 150000
working on text no 160000
working on text no 170000
working on text no 180000
working on text no 190000
working on text no 200000
CPU times: total: 2h 21min 27s
Wall time: 11h 34min 46s


## Keep only meaningful tokens and save

In [68]:
# df_sub=df.head(10000)
df['tokes']=tokens
df['postags']=postags

In [4]:
df.to_csv('data/interim/stenos_cleaned_topics_vecs.csv')

## Read in data and clean

In [6]:
df=pd.read_csv('data/interim/stenos_cleaned_topics_vecs.csv')

In [10]:
df['tokes']=df.tokes.apply(literal_eval)

In [11]:
df['postags']=df.postags.apply(literal_eval)

In [12]:
def clean_toks_postags(tokens, postags, remove_postags=['Z'], min_token_len2keep=2, print_every=1000):
    tokens_clean=[]
    postags_clean=[]
    for i, sentence in enumerate(tokens):
        if i%print_every==0:
            print(f'working on text {i}')
        toks_sent=[]
        postags_sent=[]
        for j, token in enumerate(tokens[i]):
            if len(token)>=min_token_len2keep and postags[i][j] not in remove_postags:
                toks_sent.append(token)
                postags_clean.append(postags[i][j])
        tokens_clean.append(toks_sent)
        postags_clean.append(postags_sent)
    return tokens_clean, postags_clean
        

In [14]:
%time tokens_clean, postags_clean=clean_toks_postags(df.tokes, df.postags)

working on text 0
working on text 1000
working on text 2000
working on text 3000
working on text 4000
working on text 5000
working on text 6000
working on text 7000
working on text 8000
working on text 9000
working on text 10000
working on text 11000
working on text 12000
working on text 13000
working on text 14000
working on text 15000
working on text 16000
working on text 17000
working on text 18000
working on text 19000
working on text 20000
working on text 21000
working on text 22000
working on text 23000
working on text 24000
working on text 25000
working on text 26000
working on text 27000
working on text 28000
working on text 29000
working on text 30000
working on text 31000
working on text 32000
working on text 33000
working on text 34000
working on text 35000
working on text 36000
working on text 37000
working on text 38000
working on text 39000
working on text 40000
working on text 41000
working on text 42000
working on text 43000
working on text 44000
working on text 45000
w

In [16]:
len(df.tokes), len(tokens_clean)

(209949, 209949)

## Train word2vec

In [17]:
%time model_w2v = gensim.models.Word2Vec(tokens_clean, min_count=3, vector_size=300, epochs=20)

INFO:word2vec.py:578: collecting all words and their counts
INFO:word2vec.py:561: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:word2vec.py:561: PROGRESS: at sentence #10000, processed 1070611 words, keeping 81269 word types
INFO:word2vec.py:561: PROGRESS: at sentence #20000, processed 2117165 words, keeping 121643 word types
INFO:word2vec.py:561: PROGRESS: at sentence #30000, processed 3054401 words, keeping 148212 word types
INFO:word2vec.py:561: PROGRESS: at sentence #40000, processed 4133569 words, keeping 177154 word types
INFO:word2vec.py:561: PROGRESS: at sentence #50000, processed 5113664 words, keeping 198903 word types
INFO:word2vec.py:561: PROGRESS: at sentence #60000, processed 6095760 words, keeping 218081 word types
INFO:word2vec.py:561: PROGRESS: at sentence #70000, processed 7049228 words, keeping 234905 word types
INFO:word2vec.py:561: PROGRESS: at sentence #80000, processed 8089641 words, keeping 253201 word types
INFO:word2vec.py:561: PROGRES

INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 80.43% examples, 900346 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 85.63% examples, 901662 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 90.82% examples, 901040 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 95.80% examples, 900813 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 2 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 1 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 0 more threads
INFO:word2vec.py:1629: EPOCH - 2 : training on 21348186 raw words (17976805 effective words) took 19.9s, 902482 effective words/s
INFO:word2vec.py:1585: EPOCH 3 - PROGRESS: at 4.93% examples, 924134 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 3 - PROGRESS: at 10.05% examples, 928909 words/s, in_qsize 5, out_qsize 0
INFO:word2

INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 18.99% examples, 862214 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 24.26% examples, 872174 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 29.65% examples, 878904 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 34.60% examples, 874038 words/s, in_qsize 5, out_qsize 1
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 38.50% examples, 852631 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 42.73% examples, 845110 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 47.07% examples, 842761 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 52.41% examples, 850507 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 57.08% examples, 854282 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 6 - PROGRESS: at 61.69% examples, 854134 wor

INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 60.71% examples, 914360 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 66.00% examples, 915809 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 71.27% examples, 912980 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 76.65% examples, 912548 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 81.33% examples, 912790 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 86.47% examples, 913654 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 92.09% examples, 914550 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 9 - PROGRESS: at 96.83% examples, 912535 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 2 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 1 more threads
INFO:word2vec.

INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 4.72% examples, 881030 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 9.43% examples, 879323 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 14.53% examples, 863834 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 18.78% examples, 848521 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 23.31% examples, 834654 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 27.98% examples, 831970 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 32.67% examples, 825945 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 37.06% examples, 821474 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 41.22% examples, 816916 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 13 - PROGRESS: at 45.34% examples, 80

INFO:word2vec.py:1585: EPOCH 15 - PROGRESS: at 95.75% examples, 814284 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 2 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 1 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 0 more threads
INFO:word2vec.py:1629: EPOCH - 15 : training on 21348186 raw words (17976374 effective words) took 22.1s, 812563 effective words/s
INFO:word2vec.py:1585: EPOCH 16 - PROGRESS: at 4.11% examples, 742269 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 16 - PROGRESS: at 7.85% examples, 737140 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 16 - PROGRESS: at 12.41% examples, 750379 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 16 - PROGRESS: at 16.61% examples, 747174 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 16 - PROGRESS: at 20.94% examples, 752777 words/s, in_qsize 5, out_qsize 0
INFO

INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 82.31% examples, 775552 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 86.94% examples, 778751 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 91.69% examples, 779318 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 96.19% examples, 781571 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 2 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 1 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 0 more threads
INFO:word2vec.py:1629: EPOCH - 18 : training on 21348186 raw words (17977434 effective words) took 22.9s, 784300 effective words/s
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 4.40% examples, 814217 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 8.72% examples, 809921 words/s, in_qsize 5, out_qsize 0
INFO

## Save vectors

In [18]:
word_vectors = model_w2v.wv
word_vectors.save("wordvectors/riigikogu_word2vec.wordvectors")

INFO:utils.py:447: KeyedVectors lifecycle event {'fname_or_handle': 'wordvectors/riigikogu_word2vec.wordvectors', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-09-25T09:54:49.735589', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'saving'}
INFO:utils.py:677: storing np array 'vectors' to wordvectors/riigikogu_word2vec.wordvectors.vectors.npy
INFO:utils.py:617: saved wordvectors/riigikogu_word2vec.wordvectors


## Train fasttext

In [19]:
%time model_ft = gensim.models.FastText(vector_size=300, window=5, min_count=3, sentences=tokens_clean, epochs=20)

INFO:word2vec.py:578: collecting all words and their counts
INFO:word2vec.py:561: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:word2vec.py:561: PROGRESS: at sentence #10000, processed 1070611 words, keeping 81269 word types
INFO:word2vec.py:561: PROGRESS: at sentence #20000, processed 2117165 words, keeping 121643 word types
INFO:word2vec.py:561: PROGRESS: at sentence #30000, processed 3054401 words, keeping 148212 word types
INFO:word2vec.py:561: PROGRESS: at sentence #40000, processed 4133569 words, keeping 177154 word types
INFO:word2vec.py:561: PROGRESS: at sentence #50000, processed 5113664 words, keeping 198903 word types
INFO:word2vec.py:561: PROGRESS: at sentence #60000, processed 6095760 words, keeping 218081 word types
INFO:word2vec.py:561: PROGRESS: at sentence #70000, processed 7049228 words, keeping 234905 word types
INFO:word2vec.py:561: PROGRESS: at sentence #80000, processed 8089641 words, keeping 253201 word types
INFO:word2vec.py:561: PROGRES

INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 25.38% examples, 115047 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 26.12% examples, 115024 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 26.72% examples, 114634 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 27.40% examples, 114656 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 28.09% examples, 114936 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 28.67% examples, 114805 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 29.43% examples, 114802 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 30.25% examples, 114872 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 30.84% examples, 114644 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 31.65% examples, 114841 wor

INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 79.33% examples, 114978 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 79.94% examples, 114934 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 80.60% examples, 114964 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 81.31% examples, 114995 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 82.06% examples, 115066 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 82.77% examples, 115089 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 83.35% examples, 115086 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 84.04% examples, 115083 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 84.80% examples, 115135 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 1 - PROGRESS: at 85.54% examples, 115252 wor

INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 35.90% examples, 125173 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 36.66% examples, 125269 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 37.34% examples, 125380 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 38.16% examples, 125535 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 38.86% examples, 125512 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 39.54% examples, 125418 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 40.23% examples, 125503 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 40.90% examples, 125525 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 41.70% examples, 125643 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 42.48% examples, 125492 wor

INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 94.84% examples, 125557 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 95.45% examples, 125483 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 96.19% examples, 125539 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 96.91% examples, 125509 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 97.64% examples, 125602 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 98.44% examples, 125583 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 99.09% examples, 125528 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 2 - PROGRESS: at 99.78% examples, 125505 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 2 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 1 more threads
INFO:word2vec.

...

INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 32.14% examples, 135248 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 32.96% examples, 135376 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 33.84% examples, 135452 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 34.69% examples, 135544 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 35.47% examples, 135697 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 36.24% examples, 135909 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 37.02% examples, 135983 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 37.77% examples, 136065 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 38.57% examples, 136071 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 39.41% examples, 

INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 95.88% examples, 138068 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 96.68% examples, 138078 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 97.39% examples, 138086 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 98.28% examples, 138104 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 99.00% examples, 138100 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 18 - PROGRESS: at 99.73% examples, 138096 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 2 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 1 more threads
INFO:word2vec.py:1288: worker thread finished; awaiting finish of 0 more threads
INFO:word2vec.py:1629: EPOCH - 18 : training on 21348186 raw words (17976789 effective words) took 130.1s, 138154 effective words/s
I

INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 57.42% examples, 139361 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 58.20% examples, 139341 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 58.95% examples, 139282 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 59.69% examples, 139345 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 60.43% examples, 139324 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 61.28% examples, 139236 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 62.06% examples, 139328 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 62.89% examples, 139368 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 63.61% examples, 139334 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 19 - PROGRESS: at 64.52% examples, 

INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 17.94% examples, 136729 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 18.61% examples, 136503 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 19.44% examples, 136550 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 20.29% examples, 136786 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 21.23% examples, 136993 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 21.99% examples, 136975 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 22.84% examples, 137378 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 23.57% examples, 137296 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 24.42% examples, 137387 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 25.29% examples, 

INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 81.27% examples, 137951 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 82.15% examples, 137988 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 82.94% examples, 138000 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 83.80% examples, 138039 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 84.72% examples, 138105 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 85.54% examples, 138165 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 86.23% examples, 138145 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 86.98% examples, 138138 words/s, in_qsize 5, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 87.86% examples, 138184 words/s, in_qsize 6, out_qsize 0
INFO:word2vec.py:1585: EPOCH 20 - PROGRESS: at 88.70% examples, 

## Save vectors

In [20]:
word_vectors_ft = model_ft.wv
word_vectors_ft.save("wordvectors/riigikogu_ft.wordvectors")

INFO:utils.py:447: FastTextKeyedVectors lifecycle event {'fname_or_handle': 'wordvectors/riigikogu_ft.wordvectors', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-09-25T11:58:41.305986', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'saving'}
INFO:utils.py:709: not storing attribute vectors
INFO:utils.py:677: storing np array 'vectors_vocab' to wordvectors/riigikogu_ft.wordvectors.vectors_vocab.npy
INFO:utils.py:677: storing np array 'vectors_ngrams' to wordvectors/riigikogu_ft.wordvectors.vectors_ngrams.npy
INFO:utils.py:709: not storing attribute buckets_word
INFO:utils.py:617: saved wordvectors/riigikogu_ft.wordvectors


## Example similar words

In [21]:
word_vectors_ft.most_similar('kool')

[('koolkond', 0.8596926331520081),
 ('e-kool', 0.8544713854789734),
 ('keskkool', 0.8158486485481262),
 ('koolilaps', 0.8155820965766907),
 ('koolipoiss', 0.8018566370010376),
 ('koolibuss', 0.7951183915138245),
 ('balletikool', 0.7884907722473145),
 ('kunstikool', 0.7793153524398804),
 ('algkool', 0.7788565754890442),
 ('koolivend', 0.7765803933143616)]

In [22]:
word_vectors.most_similar('kool')

[('gümnaasium', 0.6466294527053833),
 ('lasteaed', 0.634321391582489),
 ('kooli', 0.6063193678855896),
 ('õpilane', 0.603509247303009),
 ('põhikool', 0.5938141345977783),
 ('lapsevanem', 0.5824156999588013),
 ('laps', 0.5646920204162598),
 ('koolijuht', 0.5638033747673035),
 ('erakool', 0.5614989995956421),
 ('kogukond', 0.555803656578064)]